In [ ]:
# jupyter notebook to generate composite 2d frames consisting of density, ergo regions, angular deficit, and data.

In [1]:
import os, glob
import numpy as np
from PIL import Image, ImageFont, ImageDraw
from matplotlib.mathtext import math_to_image
from matplotlib import font_manager

In [2]:
# Some functions

def load_images(step):
    # Upload images 
    file_names = ['frames-2d/cosmic-string-2d.00{:02}.jpg'.format(step), \
                  'deficit-angle-plots/deficit_angle_0{:02}.jpg'.format(step), \
                  'highlighted-dataplots/highlighted_data_0{:02}.jpg'.format(step)]
    cwd = os.getcwd()
    image_list = [os.path.abspath(os.path.join(cwd, file_name)) for file_name in file_names]

    return [ Image.open(i) for i in image_list ]

def change_height_proportionally(img, width):
    # Change height of image proportional to given width.
    wpercent = width / img.size[0]
    proportional_height = int(img.size[1] * wpercent)
    return img.resize((width, proportional_height), Image.ANTIALIAS)

def change_width_proportionally(img, height):
    # Change height of image proportional to given width.
    hpercent = height / img.size[1]
    proportional_width = int(img.size[0] * hpercent)
    return img.resize((proportional_width, height), Image.ANTIALIAS)

def make_same_width(images):
    # Make all images in input list the same width.
    min_width = min([i.size[0] for i in images])
    resized = [change_height_proportionally(img, min_width) for img in images]
    return [np.asarray(i) for i in resized]

def make_same_height(images):
    # Make all images in input list the same width.
    min_height = min([i.size[1] for i in images])
    resized = [change_width_proportionally(img, min_height) for img in images]
    return [np.asarray(i) for i in resized]

#def add_label(img, E0val):
#    # Add label annotation to hardcoded locations.
#    font = ImageFont.truetype("/Library/Fonts/Georgia Bold.ttf", size=36, encoding="unic")
#    draw = ImageDraw.Draw(img)
#    draw.text((50, 50), "E0 = {:}".format(E0val), (255, 255, 255), font=font) #, size=40)

def find_E0val(step):
    
    # Read E0 from ppdata.csv
    cwd = os.getcwd()
    data_file = os.path.abspath(os.path.join(cwd, os.pardir, 'step_0{:02}/ppdata.csv'.format(step)))
    header = np.genfromtxt(data_file, max_rows=1, delimiter=',', dtype=str).tolist()
    E0val = np.genfromtxt(data_file, delimiter=',', skip_header=1, dtype = float, \
                          usecols=(header.index('E0')), unpack=True)

    return np.asscalar(E0val)

def generate_label(step):
    
    # Find E0 values 
    E0val = find_E0val(step)
    
    # Create label image
    properties = font_manager.FontProperties(size=36)
    math_to_image("$E_0$ = {:04.3f}".format(E0val), "E0label.png", prop=properties, dpi=1000, format='png')

    # Flip black and white
    label_img_arr = np.asarray(Image.open("E0label.png"))[:,:,:3]
    mod_label_img_arr = 255*np.ones_like(label_img_arr) - label_img_arr
    label_img = Image.fromarray(mod_label_img_arr)
    
    return label_img
    
def add_label(img, label_img):
    # Add label annotation to hardcoded locations.
    
    # Find midpoint and rescale label_img
    quarterpoint = int(img.size[0]/4) # midpoint of background
    scaled_label = change_height_proportionally(label_img, quarterpoint)
    
    return img.paste(scaled_label, (int(1.5*quarterpoint), 50))

In [5]:
steps = [int(name.split('.')[2]) for name in glob.glob('./frames-2d/cosmic-string-2d.*')]

for step in steps:
    
    # Load images
    imgs = load_images(step)

    # Resize images
    data_row_arr = np.hstack(make_same_height(imgs[1:3]))
    data_row_img = Image.fromarray(data_row_arr)

    scaled_string_arr = np.asarray(change_height_proportionally(imgs[0], data_row_img.size[0]))
    scaled_string_img = Image.fromarray(scaled_string_arr)

    # Form composite
    final_composite = Image.fromarray(np.vstack([scaled_string_arr, data_row_arr]))
    
    # Generate label and combine with composite plot
    add_label(final_composite, generate_label(step))
    
    # Save file
    save_file = './composite-2d-frames/composite_frame_00{:02}.jpg'.format(step)
    print('Saving composite image file to {:}'.format(save_file))
    final_composite.save(save_file, "JPEG")


os.remove("E0label.png")    

Saving composite image file to ./composite-2d-frames/composite_frame_0000.jpg
Saving composite image file to ./composite-2d-frames/composite_frame_0001.jpg
Saving composite image file to ./composite-2d-frames/composite_frame_0002.jpg
Saving composite image file to ./composite-2d-frames/composite_frame_0003.jpg
Saving composite image file to ./composite-2d-frames/composite_frame_0004.jpg
Saving composite image file to ./composite-2d-frames/composite_frame_0005.jpg
Saving composite image file to ./composite-2d-frames/composite_frame_0006.jpg
Saving composite image file to ./composite-2d-frames/composite_frame_0007.jpg
Saving composite image file to ./composite-2d-frames/composite_frame_0008.jpg
Saving composite image file to ./composite-2d-frames/composite_frame_0009.jpg
Saving composite image file to ./composite-2d-frames/composite_frame_0010.jpg
Saving composite image file to ./composite-2d-frames/composite_frame_0011.jpg
Saving composite image file to ./composite-2d-frames/composite_f